In [130]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np

import libry as ry
import time
print(cv.__version__)

4.5.3


In [131]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/ex05_redball.g")
D = RealWorld.view()

ball = RealWorld.getFrame("ball")

In [132]:
#RealWorld.addFrame(camera_fixpoint)

In [133]:
# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.bullet, verbose=2)
S.addSensor("camera")

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:68(0) starting bullet engine ...

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:81(0) ... done starting bullet engine
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:68(0) starting bullet engine ...

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:85(0) creating Configuration within bullet ...
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:81(0) ... done starting bullet engine

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:addLink:259(0) adding link anchored at 'world' as static** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:85(0) creating Configuration within bullet ...


**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:addLink:259(0) addin

In [134]:
# we're adding an "imp" to the simulation, which is a little process that can inject perturbations
#S.addImp(ry.ImpType.objectImpulses, ['obj0'], [])

In [135]:
# create your model world
C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
cameraFrame = C.frame("camera")
D2 = C.view()

#the focal length
f = 0.895
f = f * 360.
fxfypxpy = [f, f, 320., 180.]

In [136]:

config_obj_name = "conf_ball"

# delete if necceary
#C.delFrame(config_obj_name)

obj = C.addFrame(config_obj_name,"camera")

obj.setShape(ry.ST.sphere, [.03])
obj.setColor([1.,0,0])

objectPosition = ball.getPosition()
#set the model object to percept
obj.setPosition(objectPosition)



In [137]:

tau = .01

gripping = False
grasped = False


for t in range(600):
    time.sleep(0.005)
    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
        cameraFrame.setPointCloud(points, rgb)

        #if len(rgb)>0: cv.imshow('OPENCV - rgb', rgb)
        #if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)

    
    #TOTAL CHEAT: grab the true position from the RealWorld
    objectPosition = ball.getPosition()

    #set the model object to percept
    obj.setPosition(objectPosition)

    C.setJointState(q) #set your robot model to match the real q

    diff_y,diff_J = C.evalFeature(ry.FS.positionDiff,["R_gripper",config_obj_name])
    vec_y,vec_J = C.evalFeature(ry.FS.scalarProductXZ,["R_gripper","world"])


    #stack all tasks
    y = np.block([1e1*diff_y, vec_y])
    J = np.block([[1e1*diff_J],[vec_J]])

    #[y,J] = C.evalFeature(ry.FS.poseDiff, ["R_gripper", "marker"])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y);
    
    #vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ y



    if(not gripping and (np.linalg.norm(diff_y) < .04)):
        S.closeGripper("R_gripper",width=0.05, speed = 1.)
        gripping = True
    

    if S.getGripperIsGrasping("R_gripper"): print("Trueeeeeee")

    if(gripping and S.getGripperIsGrasping("R_gripper")):
        print("GRASPED!******************************************************++++")
        break


    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)



** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:196
** optimization time:0.001668 (kin:7.8e-05 coll:0 feat:0.000707 newton: 0.000204) setJointStateCount:19
   sos:54.4619 ineq:0 eq:0.0669574
**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:closeGripper:312(1) closing gripper without near object (if this is not what you expect, did you setContact(1) for the object you want to grasp?)

** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:closeGripper:312(1) closing gripper without near object (if this is not what you expect, did you setContact(1) for the object you want to grasp?)
**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:modConfiguration:568(1) terminating closing gripper (limit) - nothing grasped

** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:modConfiguration:568(1) terminating closing gripper (limit) - nothing g

In [138]:
# calculate a lift of the ball

# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,20, 3., True)
komo.addObjective([1.], ry.FS.position, ["R_gripper"], ry.OT.sos, [1e2],[-.0,.1,1.6])
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)
komo.addObjective([1.], 
                    ry.FS.vectorY, 
                    ["R_gripper"], 
                    ry.OT.sos, 
                    [1e2],
                    [0.,0.,-1]);
komo.optimize()
#komo.getReport()
#V = komo.view_play(False, 1)


** KOMO::run solver:sparse collisions:1 x-dim:280 T:20 k:2 phases:1 stepsPerPhase:20 tau:0.15  #timeSlices:22 #totalDOFs:280 #frames:2156
** optimization time:0.081035 (kin:0.001897 coll:0.063427 feat:0.006855 newton: 0.003429) setJointStateCount:15
   sos:1.77954 ineq:0 eq:1.69337e-05


In [139]:

# here loop for upwards move
#input()
for frame,tau in zip(komo.getPathFrames(),komo.getPathTau()):
    time.sleep(tau)



    [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
    
    C.setFrameState(frame)

    q= C.getJointState()
    
    #send position to the simulation
    S.step(q, tau, ry.ControlMode.position)

tau = 0.01
for t in range(100):
    time.sleep(tau)

    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
    
        #if len(rgb)>0: cv.imshow('OPENCV - rgb', rgb)
        #if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)
    
    if S.getGripperWidth("R_gripper") < 0.02:
        S.openGripper("R_gripper",speed=3.)

    S.step([], tau, ry.ControlMode.none)

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:248(1) gripper 'R_gripper' does not hold an object

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:264(1) initiating opening gripper R_gripper
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:248(1) gripper 'R_gripper' does not hold an object

** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:264(1) initiating opening gripper R_gripper
**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:248(1) gripper 'R_gripper' does not hold an object

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:264(1) initiating opening gripper R_gripper
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:248(1) gripper 'R_gripper' does not hold an object

** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.